In [1]:
import numpy as np
from pymatgen.io.cif import CifParser
from pymatgen.core import Structure
from pymatgen.core.lattice import Lattice

In [2]:
import sys
sys.path.append('../..')
from ai4mat.data.data import (
    StorageResolver)

In [3]:
from ase.visualize import view
from pymatgen.io.ase import AseAtomsAdaptor

In [4]:
mp_cell = CifParser(StorageResolver()["materials"] / 
    "GaSe_mp-1572_computed.cif").get_structures(primitive=False)[0]

/home/kna/.cache/pypoetry/virtualenvs/2d-defects-potential-learning-pYjw2mkT-py3.10/lib64/python3.10/site-packages/pymatgen/io/cif.py:1155: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [5]:
selection = (mp_cell.frac_coords[:, 2] < 0.7) & (mp_cell.frac_coords[:, 2] > 0.3)
assert np.count_nonzero(selection)*2 == len(mp_cell)

In [6]:
view(AseAtomsAdaptor.get_atoms(mp_cell), viewer='ngl')

In [7]:
# Create a new Structure object with the selected atoms
monolayer = Structure.from_sites([mp_cell[i] for i in np.where(selection)[0]])
lattice_np = np.array(monolayer.lattice.matrix)
lattice_np[2, 2] = 20
tall_lattice = Lattice(lattice_np)

reference_cell = Structure(
    lattice=tall_lattice,
    species=monolayer.species,
    coords=monolayer.cart_coords,
    coords_are_cartesian=True,
)

In [8]:
reference_cell

Structure Summary
Lattice
    abc : 3.81906671 3.81906671 20.0
 angles : 90.0 90.0 120.00001238000002
 volume : 252.62426458036785
      A : 3.81906671 0.0 2.3385039110658565e-16
      B : -1.9095340696375986 3.3074083770111447 2.3385039110658565e-16
      C : 0.0 0.0 20.0
    pbc : True True True
PeriodicSite: Ga (1.9095, 1.1025, 7.6655) [0.6667, 0.3333, 0.3833]
PeriodicSite: Ga (1.9095, 1.1025, 10.1363) [0.6667, 0.3333, 0.5068]
PeriodicSite: Se (-0.0000, 2.2049, 11.3096) [0.3333, 0.6667, 0.5655]
PeriodicSite: Se (-0.0000, 2.2049, 6.4922) [0.3333, 0.6667, 0.3246]

In [9]:
from pymatgen.io.cif import CifWriter
CifWriter(reference_cell).write_file(StorageResolver()["materials"] /"GaSe.cif")

Structure appears to have two layers https://www.researchgate.net/figure/Basic-properties-of-InSe-monolayer-a-Top-and-side-view-of-InSe-monolayer-b-Band_fig1_324591452

In [10]:
supercell = reference_cell.copy()
supercell.make_supercell((6, 6, 1))

In [11]:
view(AseAtomsAdaptor.get_atoms(supercell), viewer='ngl')

In [12]:
mp_cell

Structure Summary
Lattice
    abc : 3.81906671 3.81906671 17.801796
 angles : 90.0 90.0 120.00001238000002
 volume : 224.8582811354867
      A : 3.81906671 0.0 2.3385039110658565e-16
      B : -1.9095340696375986 3.3074083770111447 2.3385039110658565e-16
      C : 0.0 0.0 17.801796
    pbc : True True True
PeriodicSite: Ga (0.0000, 0.0000, 1.2356) [0.0000, 0.0000, 0.0694]
PeriodicSite: Ga (0.0000, 0.0000, 16.5662) [0.0000, 0.0000, 0.9306]
PeriodicSite: Ga (1.9095, 1.1025, 7.6655) [0.6667, 0.3333, 0.4306]
PeriodicSite: Ga (1.9095, 1.1025, 10.1363) [0.6667, 0.3333, 0.5694]
PeriodicSite: Se (1.9095, 1.1025, 15.3932) [0.6667, 0.3333, 0.8647]
PeriodicSite: Se (1.9095, 1.1025, 2.4086) [0.6667, 0.3333, 0.1353]
PeriodicSite: Se (-0.0000, 2.2049, 11.3096) [0.3333, 0.6667, 0.6353]
PeriodicSite: Se (-0.0000, 2.2049, 6.4922) [0.3333, 0.6667, 0.3647]

In [13]:
len(supercell)/2* 0.062501

4.500072

In [14]:
len(supercell)/2

72.0

In [15]:
0.125/2

0.0625

In [16]:
0.125/2

0.0625

In [17]:
5/(len(supercell)/2)

0.06944444444444445